# A股验证后股票数据 V1 （202409）

# Validated Stock Data of A-share Stock In Chinese Market V1 （202409）

## 1. A股清单和CDA验证
我们采用sqlite数据库保存A股清单
### 1.1 AkShareA股清单下载
下载所有AKShare股票A股清单
（通过实时行情数据-京沪深A股栏目，提取 https://akshare.akfamily.xyz/data/stock/stock.html#id1 ）

In [1]:
import pandas as pd
import sqlite3
import akshare as ak
import datetime
import numpy as np
import logging
import json

#disable future warnings of akshare package
import warnings
warnings.filterwarnings("ignore")

filelink='Data/stock.db'

# derived from CQF E3 Assginment, delete the trans frunction.
# function of data acquisition: if there's table in database use 
#     native data, else retrieve from akShare Source.
# AKShareFunc -- AKShare function to retrieve data from web source,
# paraDict -- parameters of AKShare function in the form of Dictionary,
# tablename -- table name to store data(svm.db),
# filelink -- database file link,
def get_data(AKShareFunc,paraDict,tablename,filelink=filelink,inform=True):
    conn = sqlite3.connect(filelink)
    cursor = conn.cursor()
    cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table'\
    AND name='{tablename}'")
    result = cursor.fetchone()
    dataEng=pd.DataFrame()
    if not result:
        if inform:
            print('get data from source.')
        fundData = AKShareFunc(**paraDict)
        fundData.to_sql(tablename, conn,index=False)
    else:
        if inform:
            print('read from sql.')
    funddataEng=pd.read_sql(f"SELECT * FROM {tablename}", conn)
    cursor.close()
    conn.close()
    return funddataEng

#AkShare realtime quotes
AKShareFunc=ak.stock_zh_a_spot_em
paraDict={}
tablename='stockSymb'
AKrealtimeA = get_data(AKShareFunc,paraDict,tablename)
AKrealtimeA

read from sql.


,序号,代码,名称,最新价,涨跌幅,涨跌额,成交量,成交额,振幅,最高,...,量比,换手率,市盈率-动态,市净率,总市值,流通市值,涨速,5分钟涨跌,60日涨跌幅,年初至今涨跌幅
0,1,300990,同飞股份,28.43,20.01,4.74,36161.0,1.028044e+08,0.00,28.43,...,16.99,8.59,108.67,2.79,4.789886e+09,1.197472e+09,0.00,0.00,-0.63,-35.40
1,2,301421,波长光电,35.88,20.00,5.98,123277.0,4.182630e+08,14.05,35.88,...,11.80,26.62,72.09,3.53,4.151962e+09,1.661548e+09,0.00,0.00,-18.27,-42.25
2,3,300293,蓝英装备,16.21,12.02,1.74,384030.0,6.324526e+08,10.85,17.30,...,6.82,13.70,-2228.28,6.11,5.486082e+09,4.544245e+09,-0.25,2.27,12.88,10.27
3,4,300225,金力泰,5.82,11.71,0.61,198070.0,1.127429e+08,17.08,6.10,...,26.54,4.18,106.47,3.31,2.767000e+09,2.760715e+09,0.34,0.69,20.00,-40.49
4,5,688502,茂莱光学,82.00,11.02,8.14,8338.0,6.626668e+07,6.54,82.00,...,10.80,5.14,139.02,3.77,4.329600e+09,1.330555e+09,2.62,4.23,-16.17,-62.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5639,5640,300030,阳普医疗,5.61,-15.38,-1.02,452528.0,2.568731e+08,15.54,6.33,...,3.59,16.62,-164.40,2.42,1.734541e+09,1.527173e+09,-2.43,-1.41,54.55,-27.61
5640,5641,300746,汉嘉设计,9.55,-16.15,-1.84,240949.0,2.364826e+08,14.49,10.76,...,9.34,10.86,289.74,1.82,2.155801e+09,2.118994e+09,-0.21,0.53,60.50,-2.85
5641,5642,300343,联创股份,4.18,-16.57,-0.83,513037.0,2.191328e+08,8.78,4.46,...,3.43,4.81,-175.60,2.33,4.468291e+09,4.456797e+09,-0.95,-1.42,-11.06,-36.67
5642,5643,300677,英科医疗,21.70,-20.01,-5.43,76481.0,1.683744e+08,6.56,23.48,...,6.86,1.63,11.98,0.85,1.405946e+10,1.017580e+10,0.00,0.00,-15.89,-6.79


### 1.2 国泰君安富易APP所有A股清单
#### 1.2.1 沪深京盘后数据下载维护
点击“行情/A股市场”，设置/盘后数据下载（1992-1-1至今）
![gtja_afterdata.png](image/gtja_afterdata.png)

#### 1.2.2 导出数据xls
点击“行情/A股市场”，菜单选择：“设置/数据导出”（/Users/mac/Downloads/同步空间/notebook/薛猫选股/A股验证股票数据/Data/manual/全部Ａ股20240916.xls）。

#### 1.2.3 手工整理csv
形成文档"Data/gtjaA20240916.csv"，主要整理：  
- 去掉最后一行“数据来源:通达信"
- 同时检查单个或者多个0开头的股票代码是否是文本形式
- 将空值"--"转换为-1
- 将xls转换为csv
![guotaiA.png](image/guotaiA.png)

#### 1.2.4 读入数据

In [2]:
import pandas as pd
datalink='Data/gtjaA20240916.csv'
gt_A=pd.read_csv(datalink,dtype={"代码":str})
gt_A

,代码,名称,涨幅%,现价,涨跌,买价,卖价,总量,现量,涨速%,换手%,今开,最高,最低,昨收,市盈(动),总金额,量比
0,920016,N中草,241.30,25.60,18.10,25.60,25.61,153722,649,-8.56,98.36,12.49,45.00,12.20,7.50,37.25,24727.28,0.00
1,430198,微创光电,29.92,6.21,1.43,6.21,-1.00,129350,366,0.00,18.41,4.77,6.21,4.76,4.78,-1.00,7635.48,17.15
2,300635,中达安,20.05,9.82,1.64,9.82,-1.00,223227,2203,0.00,18.59,8.20,9.82,8.10,8.18,144.93,20830.57,2.20
3,300746,汉嘉设计,20.02,11.39,1.90,11.39,-1.00,405195,134,0.00,18.26,10.73,11.39,10.31,9.49,345.58,44838.12,6.39
4,300067,安诺其,20.00,4.08,0.68,4.08,-1.00,1602042,27787,0.00,17.09,3.35,4.08,3.28,3.40,211.68,60358.46,3.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5352,300134,大富科技,-16.16,10.12,-1.95,10.12,10.13,1136319,18187,-0.29,15.95,11.54,11.79,9.80,12.07,-1.00,122702.48,1.19
5353,688615,合合信息,-1.00,0.00,-1.00,-1.00,-1.00,0,0,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,0.00,-1.00
5354,603361,浙江国祥,-1.00,0.00,-1.00,-1.00,-1.00,0,0,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,0.00,-1.00
5355,603091,众鑫股份,-1.00,0.00,-1.00,-1.00,-1.00,0,0,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,-1.00,0.00,-1.00


### 1.3 A股清单数据比对
#### 1.3.1 比对函数

In [3]:
#find two difference of DataFrame column value items regardless of its sequences. 
# NOT SUITABLE FOR INDEX COMPARISON ON COLUMNS FOR SEQUENCE REDUPLICATED VALUES
# colnameList: List of the names of columns to be compared
# return 
#  colname:
#  df12common:
#  df22common:
# edited 2024-10-7 can be 
def compareDiff(df1,df2,colnameList=None):
    diffDf=pd.DataFrame()
    if colnameList==None:
        colnameList=list(set(df1.columns.tolist()+df2.columns.tolist()))
    for colname in colnameList:
        df1Arr=set(df1[colname])
        df2Arr=set(df2[colname])

        common=df1Arr & df2Arr
        df12common=(len(df1Arr)-len(common))/len(df1Arr)
        df22common=(len(df2Arr)-len(common))/len(df2Arr)
        
        df1todf2=df1Arr-df2Arr
        df2todf1=df2Arr-df1Arr
        
        #change decimals to strings if values are numbers
        if len(df1todf2)>0 and type(list(df1todf2)[0])!='str':
            df1todf2=set(map(str,df1todf2))
        if len(df2todf1)>0 and type(list(df2todf1)[0])!='str':
            df2todf1=set(map(str,df2todf1))
            
        df1Diff=','.join(df1todf2)
        df2Diff=','.join(df2todf1)
        #print(df1Diff,df2Diff)
        
        # colname: name of compared columns
        # df12common,df22common: Difference ratio with intersection of DataFrames(df1,df2)
        # dif1Diff,dif2Diff: List of Differences
        newdf=pd.DataFrame({
            'colname':[colname],
            'df12common':[df12common],
            'df22common':[df22common],
            'df1Diff':[df1Diff],
            'df2Diff':[df2Diff]
        })
        diffDf=pd.concat([diffDf,newdf],ignore_index=True)

    return diffDf

#### 1.3.2 AkShare和国泰君安A股清单差异分析
国泰君安与AkShare数据公共部分：common  
国泰君安数据与公共部分完全重合，差异为：0%  
AkShare数据与公共部分差异：5.09%  
结论：AkShare数据比国泰君安多出287条。

In [4]:
AkGtjaDifDf=compareDiff(gt_A,AKrealtimeA,['代码'])
AkGtjaDifDf

,colname,df12common,df22common,df1Diff,df2Diff
0,代码,0.0,0.05085,,"300392,600291,000699,000730,600786,300202,6006..."


AkShare数据的差异部分列表(比国泰君安多出的股票)

In [5]:
AkGtjaDifList=AkGtjaDifDf['df2Diff'].tolist()[0].split(',')
df_filtered = AKrealtimeA[AKrealtimeA['代码'].isin(AkGtjaDifList)]
df_filtered

,序号,代码,名称,最新价,涨跌幅,涨跌额,成交量,成交额,振幅,最高,...,量比,换手率,市盈率-动态,市净率,总市值,流通市值,涨速,5分钟涨跌,60日涨跌幅,年初至今涨跌幅
716,717,920019,铜冠矿建,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,7.63,1.00,6.581600e+08,NaN,NaN,NaN,0.00,0.00
718,719,833994,翰博高新,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,-46.49,5.32,6.133712e+09,4.855600e+09,NaN,NaN,0.00,0.00
719,720,833874,泰祥股份,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,46.14,3.52,2.010987e+09,9.635979e+08,NaN,NaN,2.55,2.55
720,721,832317,观典防务,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,-98.58,8.68,8.106881e+09,8.106881e+09,NaN,NaN,0.00,0.00
723,724,688555,退市泽达,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1103,1104,000018,神城A退,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00
1104,1105,000015,PT中浩A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00
1105,1106,000013,*ST石化A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00
1106,1107,000005,ST星源,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.00,-22.43


这些股票都没有最新价信息（即AkShare比国泰君安多出部分的股票清单与没有实时报价信息的股票清单重合————即国泰君安股票清单与AkShare股票清单在具备实时报价信息上一致）。

In [6]:
df_filtered2=df_filtered[df_filtered['最新价'].isna()]  #stocks without current quotes
AkDifAkNulldf=compareDiff(df_filtered,df_filtered2,['代码'])
AkDifAkNulldf

,colname,df12common,df22common,df1Diff,df2Diff
0,代码,0.0,0.0,,


#### 1.3.3 补充说明：AkShare 两网和退市股票并不包括全部退市股票
AkShare溢出的股票清单并不在 AkShare 两网及退市股票信息中

In [7]:
import akshare as ak

#AkShare delisted stocks
AKShareFunc=ak.stock_zh_a_stop_em
paraDict={}
tablename='delist'
AKdelist = get_data(AKShareFunc,paraDict,tablename)[['代码','名称']]
 
filtereddf3=compareDiff(df_filtered,AKdelist,['代码'])
filtereddf3

read from sql.


,colname,df12common,df22common,df1Diff,df2Diff
0,代码,1.0,1.0,"600291,300392,000699,000730,600786,300202,6006...","400116,400188,400005,400079,400099,400185,4000..."


### 1.4 A股清单做基本面信息验证
AkShare数据有基本面信息接口，我们查询此基本面信息接口，做以下两件事情： 
- 与我们保存的A股清单的“代码”和“名称”做验证，防止股票代码信息与股票信息错位（见 [涨停板敢死队](../涨停板敢死队/涨停板敢死队.ipynb) 2.2.4.2 获得基准验证数据：国泰君安富易App2020年数据集）
- 增加IPO日期（用于校验历史数据起始日期）

对A股清单的代码和名称做交叉验证。本步骤需要人工处理，因为简称可能有所不同，但标的还是一个。代码如果需要从下文再重新运行，需要手动删除faillist.pkl文件，避免跳过检查和IPO添加环节。

In [8]:
# A is Brief of B which means:
# 1)all characters in A are in B
# 2)the order of characters in A is the same as those in B
def isBriefOf(A, B):
    lenA, lenB = len(A), len(B)
    i, j = 0, 0
    
    while i < lenA and j < lenB:
        if A[i] == B[j]:
            i += 1
        j += 1
    return i == lenA

In [9]:
import tempfile
import pickle
import numpy as np
import re
import os

# Save a temporary file named "faillist.pkl" in the current directory to support the resumption of the network query for fundamental information. 
# To start over, please manually delete the file or choose loadTemp=False.
# AShareDf: the target stock DataFrame to be checked
# addDate: add IPO date information to the DataFrame
# fix: fixed automatically if there's in accordance with the source
# loadtemp: load temp file which stores last source query, False to restart all queries
def checkFoundamental(AShareDf, addDate=True, fix=False, loadtemp=True):
    mismatch = []
    checkList = None
    faillist = []
    resdf = AShareDf
    i = 0
    failfile='faillist.pkl'
    foundfilepath='Data/found.csv'
    checkAkshare=True
    
    if loadtemp:
        if os.path.exists(failfile):
            with open(failfile, 'rb') as temp_file:
                checkList = pickle.load(temp_file)  # support resumable transfer 
            if os.path.exists(foundfilepath):
                resdf=pd.read_csv(foundfilepath)
                checkAkshare=False
        else:
            checkList = AShareDf['代码'].tolist()
    else:
        checkList = AShareDf['代码'].tolist()

    # for the stability of AKShare source, we retreat 3 times if fail
    while i < 3 and len(checkList) > 0 and checkAkshare:
        print(f'Try time {i+1}:')
        for s in checkList:
            msg = f'checking {s}...'
            print(f'{msg:<100}', end='\r')
            df_value = AShareDf.loc[AShareDf['代码'] == s, '名称'].values[0]

            querySuccess=False
            try:
                s_df = ak.stock_individual_info_em(symbol=s)
                querySuccess=True
            except Exception as e:
                msg = f'{e}. Try {s} another {2-i} times.'
                print(f'{msg:<100}', end='\r')
                faillist.append(s)

            if querySuccess:
                s_value = s_df.loc[s_df['item'] == '股票简称', 'value'].values[0]

                # delete the spaces
                df_value_no_space = re.sub(r'\s+', '', df_value)
                s_value_no_space = re.sub(r'\s+', '', s_value)

                # delete the special prefixes ('XD','N','ST','C') of stock names
                # XD: Ex-dividend and ex-rights
                # N: Newly IPO stocks the first day
                # C: Newly IPO stocks the first week
                # ST:Special Treatment
                df_value_trim = re.sub(r'^XD|^N|^ST|\*ST$|C', '', df_value_no_space)
                s_value_trim = re.sub(r'^XD|^N|^ST|\*ST$|C', '', s_value_no_space)

                # get Chinese Characters
                df_value_chinese = re.findall(r'[\u4e00-\u9fff]', df_value_trim)
                s_value_chinese = re.findall(r'[\u4e00-\u9fff]', s_value_trim)

                #whether the two strings are at least one direction inclusion relationship
                isBrief=isBriefOf(s_value_chinese,df_value_chinese) or isBriefOf(df_value_chinese,s_value_chinese)

                # 判断两个字符串相等
                if df_value_no_space == s_value_no_space or isBrief:
                    msg = f"code: {s} checked: matched codes."
                    print(f'{msg:<100}', end='\r')
                    pass
                else:
                    mismatch.append({'code': s, 'source': s_value, 'target': df_value})
                    msg = f"code: {s},df({df_value}) vs source({s_value}) mismatched."
                    if fix:
                        AShareDf.loc[AShareDf['代码'] == s,'名称'] = s_value
                        msg += "replaced."
                    print(f'{msg:<100}', end='\r')
                    
                # add IPO date to DataFrame
                if addDate:
                    ipostr = str(s_df.loc[s_df['item'] == '上市时间', 'value'].values[0])
                    resdf.loc[resdf['代码'] == s, 'IPO'] = datetime.datetime.strptime(ipostr, "%Y%m%d") if ipostr!='-' else np.nan
        print(f'faillist({len(faillist)}):{faillist}')
        checkList = faillist
        faillist = []
        i += 1
    if checkList == []:
        msg = 'All codes are checked'
        msg += ' and fixed.' if fix else '.'
    else:
        msg = f'Check the following manually because of the inquiry failure(total {len(checkList)}): \n{checkList}'
    
    if loadtemp:
        with open(failfile, 'wb') as temp_file:
            pickle.dump(checkList, temp_file)
    
    print(f'{msg:<100}', end='\r')
    
    return {'mismatch': mismatch, 'df': resdf, 'fail': checkList}

In [10]:
checkFound=checkFoundamental(gt_A)

Try time 1:
faillist(1):['603361']matched codes.                                                                
Try time 2:
faillist(1):['603361']nother 1 times.                                                               
Try time 3:
faillist(1):['603361']nother 0 times.                                                               
Check the following manually because of the inquiry failure(total 1): 


读取json否则是一个对象

In [12]:
foundfilepath='Data/found.csv'
checkFound=checkFoundamental(gt_A) if os.path.exists(foundfilepath) else checkFoundamental(gt_A)

Try time 1:
faillist(1):['603361']nother 2 times.                                                               
Try time 2:
faillist(1):['603361']nother 1 times.                                                               
Try time 3:
faillist(1):['603361']nother 0 times.                                                               
Check the following manually because of the inquiry failure(total 1): 


通过手工查看验证结果，是曾用名的差异，且通过富易App查询，股票名近期已经改为新名。我们采用可以源头覆盖本地。  
第一次核查结果两个差异：  
603800：洪田股份 vs 道森股份（[洪田股份更名公告20240920](https://baijiahao.baidu.com/s?id=1809982231624229257&wfr=spider&for=pc)）
300378：鼎捷数智 vs 鼎捷软件 （[鼎捷数智更名公告20240923](https://baijiahao.baidu.com/s?id=1810725724075456690&wfr=spider&for=pc)）

In [13]:
checkFound['mismatch']

[]

In [14]:
gt_A_val=checkFound['df']
gt_A_val.to_csv(foundfilepath)

In [15]:
gt_A_val[['代码','名称','IPO']]

,代码,名称,IPO
0,920016,N中草,2024-09-13
1,430198,微创光电,2020-07-27
2,300635,中达安,2017-03-31
3,300746,汉嘉设计,2018-05-25
4,300067,安诺其,2010-04-21
...,...,...,...
5352,300134,大富科技,2010-10-26
5353,688615,合合信息,2024-09-26
5354,603361,浙江国祥,NaT
5355,603091,众鑫股份,2024-09-20


### 1.5 验证结论：
获取可信A股清单：
- 国泰君安大智慧富易App，点击市场行情，A股后手工导出所有清单
- AkShare采用京沪深A股实时行情接口（ak.stock_zh_a_spot_em），获得验证数据
- 若国泰君安数据缺失，则检查是否这部分缺失数据的“最新价”为空，若是，则验证通过；否则，报警，验证不通过，并列出非空的股票清单。 
- 对国泰君安数据做基本面校验，手工检查代码和简称没有错位情况。
- 若验证通过，将该股票清单保存到共享stock数据库(/Users/mac/Downloads/同步空间/notebook/薛猫选股/Data/stock.db)，按照不同日期保存A股清单的快照。

集中处理函数代码如下：

In [16]:
from datetime import date
import os

#get validated AshareList from AkShare and check
# dblink: db link to store the validated AShare list
# validlink: (optional) read from validlink of gtja data (.csv) for validation
# fix: fix names from source automatically
def storeAShareList(datalink,dblink,fix=False):
    
    gt_A=pd.read_csv(datalink,dtype={"代码":str})
    
    #validate source from AkShare
    AKrealtimeA = ak.stock_zh_a_spot_em()
    
    #validate data
    res={'valid':0,'diff':'','df':pd.DataFrame()}
    AkGtjaDifDf=compareDiff(gt_A,AKrealtimeA,['代码'])
    diffrate=AkGtjaDifDf.loc[0,'df22common']
    if diffrate>0:
        print(f'There is a {diffrate:.2%} gap comparing with AKShare verification data.')
        AkGtjaDifList=AkGtjaDifDf['df2Diff'].tolist()[0].split(',')
        df_filtered = AKrealtimeA[AKrealtimeA['代码'].isin(AkGtjaDifList)]
        df_filtered2= df_filtered[df_filtered['最新价'].isna()]
        AkDifAkNulldf=compareDiff(df_filtered,df_filtered2,['代码'])
        if AkDifAkNulldf.loc[0,'df12common']==0:
            res['valid']=1
            print('All are of none quotes, pass the verification.')
        else:
            res['diff']=AkDifAkNulldf.loc[0,'df1Diff']
            print('Check res[\'diff\'] to see details of shortage.')
    else:
        res['valid']=1
        print(f'There is no data shortage from AKShare verification.')
    
    #check manually if not automatically fixed
    file_path = 'faillist.pkl'
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"{file_path} deleted.")
    else:
        print(f"{file_path} no faillist file.")
    checkfound=checkFoundamental(gt_A,fix=fix)
    
    if res['valid']==0 and fix==False:
        print('The difference of stock codes and its names from the verification source:\n',checkfound['mismatch'])
        m=''
        while m.lower()!='y' and m.lower()!='n':
            m=input('Check manually if the names pass the verification in accordance with the source(y/n)?')
            if m.lower()=='n':
                res['valid']=0
            elif m.lower()!='y':
                print('Only support press \'y\' or \'n\', press again.')
            else:
                res['valid']=1
                
    #store to database    
    if res['valid']==1 or fix:
        current_date = date.today().strftime("%Y%m%d")
        table_name = current_date + "Asharelist"
        conn = sqlite3.connect(dblink)
        res['df']=checkfound['df'][['代码','名称','IPO']]
        res['df'].to_sql(table_name, conn, if_exists='replace', index=False)
        conn.close()
    return res

In [17]:
dblink='/Users/mac/Downloads/同步空间/notebook/薛猫选股/Data/stock.db'
result=storeAShareList(datalink,dblink,True)   #直接覆盖检查差异结果
result['df']

There is a 5.19% gap comparing with AKShare verification data.
Check res['diff'] to see details of shortage.
faillist.pkl deleted.
Try time 1:
faillist(1):['603361']matched codes.                                                                
Try time 2:
faillist(1):['603361']nother 1 times.                                                               
Try time 3:
faillist(1):['603361']nother 0 times.                                                               
Check the following manually because of the inquiry failure(total 1): 


,代码,名称,IPO
0,920016,N中草,2024-09-13
1,430198,微创光电,2020-07-27
2,300635,中达安,2017-03-31
3,300746,汉嘉设计,2018-05-25
4,300067,安诺其,2010-04-21
...,...,...,...
5352,300134,大富科技,2010-10-26
5353,688615,合合信息,2024-09-26
5354,603361,浙江国祥,NaT
5355,603091,众鑫股份,2024-09-20


In [18]:
result['df'].dtypes

代码             object
名称             object
IPO    datetime64[us]
dtype: object

In [19]:
result

{'valid': 0,
 'diff': '001279,920019,301618',
 'df':           代码    名称        IPO
 0     920016   N中草 2024-09-13
 1     430198  微创光电 2020-07-27
 2     300635   中达安 2017-03-31
 3     300746  汉嘉设计 2018-05-25
 4     300067   安诺其 2010-04-21
 ...      ...   ...        ...
 5352  300134  大富科技 2010-10-26
 5353  688615  合合信息 2024-09-26
 5354  603361  浙江国祥        NaT
 5355  603091  众鑫股份 2024-09-20
 5356  920099  瑞华技术 2024-09-25
 
 [5357 rows x 3 columns]}

## 2. A股股票历史数据

我们采用时序数据库 DolphinDB 来保存所有A股历史日数据信息。相关参考模板（https://docs.dolphindb.cn/zh/tutorials/stockdata_csv_import_demo.html）

### 2.1 获得基准验证数据：国泰君安富易App2020年数据集

通过国泰君安富易APP手工批量导出日线功能，实现全量A股有限日期导出(批量导出功能受限于2020年开始数据)。由于有限日期，作为校验线上数据AkShare使用。
注意由于复权方法各软件存在差异，为避免校验出现偏差，我们一律采用不复权的价格和成交量数据。通过国泰君安富易App实现2020年至今数据全量A股导出csv文件（保存于Data/gtja_daily）,添加全部A股栏目，加入批量导出数据。  
![export_A.png](export_A.png)

App导出的文本数据文件：
- 第一行是股票代码和名称等信息；第二行是表头，分隔符是tab（不论手工导出分隔符是什么，都是tab）；
- 有两种格式文件，遇到股票名为生僻词，是GBK形式，而其它utf-8格式。因此，需要采用两种读取引擎读取文件。

[涨停板敢死队](../涨停板敢死队/涨停板敢死队.ipynb) 中（2.2.4.2 获得基准验证数据：国泰君安富易App2020年数据集）详细记录了另一种较繁琐的方法，可以防止导出文件格式更新后，与原设定不一致的情况（目前没有发生）。为简化操作，我们采用根据数据自定义表头，跳过标题和表头与固定行跳过尾部非数据文本的方式，代码简化很多，但需要定期关注GTJA软件更新后，手工导出的数据格式变化。

In [20]:
import os
import chardet
import pandas as pd

# read gtja csv file
def readGtja(folder_path):
    df_dict={}
    
    for filename in os.listdir(folder_path):
        if filename[-4:]=='.txt':
            file_path = os.path.join(folder_path, filename)

            # judgement of encoder
            with open(file_path, 'rb') as f:
                raw_data = f.read()
                encoding = chardet.detect(raw_data)['encoding']

            #When there're uncommon characters, the txt file is GBK encoding else UTF-8, so read with either decoder.
            print(f'{filename} encoder:{encoding}',end='\r')
            if encoding=='GB2312':
                encoding='GBK'
            df = pd.read_csv(file_path, header=2, skipfooter=1,encoding=encoding)
            if not df.empty:
                names=['DateTime','Open','High','Low','Close','Volume','Amount']
                df.rename(columns=dict(zip(df.columns, names)), inplace=True)
                df['DateTime'] = pd.to_datetime(df['DateTime'])
                df.set_index('DateTime',inplace=True)
            df_dict[filename[:6]]=df
    print('finished all data reading to DataFrame.')
    return df_dict

folder_path = 'Data/gtja_daily_bfq'
validateDfs=readGtja(folder_path)

finished all data reading to DataFrame.


In [21]:
validateDfs['920016']

,Open,High,Low,Close,Volume,Amount
DateTime,,,,,,
2024-09-18,17.92,21.63,17.92,17.92,11714695,223396192.0
2024-09-19,18.40,21.40,18.20,19.41,10766790,207757856.0
2024-09-20,18.00,19.16,17.52,18.51,8673066,157461264.0
2024-09-23,18.49,18.50,16.88,17.09,7344424,127252616.0
2024-09-24,17.63,20.95,17.40,18.98,10227850,195159936.0
2024-09-25,17.50,20.18,17.45,20.10,9513892,175683024.0
2024-09-26,19.33,20.50,19.11,19.44,7487696,147338064.0
2024-10-08,36.70,36.85,31.30,36.85,8731210,303611136.0


### 2.2 DolphinDB准备：数据量预估和数据库分区规划
#### 2.2.1 分区规划预算
DolphinDB最佳分区存储量的描述有两处建议：
- 100M一个单元（[DolphinDB官网:国内股票行情数据导入实例国内股票行情数据导入实例](https://docs.dolphindb.cn/zh/tutorials/stockdata_csv_import_demo.html)
- 数据分区的文档[数据分区粒度控制](https://docs.dolphindb.com/zh/db_distr_comp/db/db_partitioning.html) 中“分区粒度不宜过小”100M下限。

以上两处关于存储的直接建议似有冲突，所以我们根据第二篇文档中:  
<center>$ 分区大小=\frac{内存}{8\times工作线程数量} $</center>

的公式，假设目前本地laptop只有8G内存，双核处理器支持自然2线程工作，取 512M作为分区单位大小，所以我们对于股票日数据量与文件大小做一个测量，然后确定分区规划。  
通过以下程序估测，得到如下数据：平均每日（行）数据量为56.42byte, 总存储量822.64M，所有A股股票总日期（行）数为14,581,369行。  

In [22]:
import codecs

# estimate the unit line storage data size of all txt files in folder_path
def estimateUnitSize(folder_path):
    
    # total rows and total file size
    total_valid_rows = 0
    total_file_size = 0

    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)

        try:
            with codecs.open(file_path, 'r', encoding='utf-8') as file:
                lines = file.readlines()
        except UnicodeDecodeError:
            with codecs.open(file_path, 'r', encoding='gbk') as file:
                lines = file.readlines()
                
        #valid rows
        valid_rows = len(lines) - 4
        total_valid_rows += valid_rows

        # filesize
        file_size = os.path.getsize(file_path)
        total_file_size += file_size

    average_size_per_row = total_file_size / total_valid_rows
    return {'unit':average_size_per_row,'size':total_file_size,'rows':total_valid_rows}

unitsizeDict=estimateUnitSize(folder_path)

In [23]:
unitsizeDict

{'unit': 54.496802900867415, 'size': 796392299, 'rows': 14613560}

#### 2.2.2 A股存储分类规划
所以，有以下估算关系：  
- 沪深A股总共股票数：5357
- txt文件总体存储量：822.64M
- 每行日数据约占56.42字节
- 总计日期数：14,581,369
- 平均每个股票日期数：2722日，11.25年（242日/年)
- 每个股票每年日数据占用数据存储量：13.66K
- 每个股票所有日数据最大(2030年-1992年=38年)占用数据存储量：518K
- 分区经验公式为：C/8W（见 [DolphinDB官网 4.2 分区粒度不宜过小](https://docs.dolphindb.com/zh/tutorials/database.html#31-%E8%8C%83%E5%9B%B4-range-%E5%88%86%E5%8C%BA)）根据目前本机 4G，1进程，约512M每个分区
- 每个分区可容纳平均股票数:986只

In [24]:
import math
#calculation
bytePerDay=unitsizeDict['unit']
bytePerYear=bytePerDay*242
YearsPerStock=2030-1992
bytePerStock=bytePerYear*YearsPerStock

unitMem=512000000
stocksPerPartition=int(unitMem/bytePerStock)

totalStocks=result['df'].shape[0]
partitionNum=math.ceil(totalStocks/stocksPerPartition)
print('stocks per partition:',stocksPerPartition,', need at least ',partitionNum,' partitions.')

stocks per partition: 1021 , need at least  6  partitions.


### 2.3 建立DolphinDB分区数据库
根据查询需求（查询特定股票的日线数据、具体日期段数据等），我们决定将股票代码（Symbol)作为数据库分区维度，时间作为数据表分区维度。  
数据表分区维度Symbol可以采用两种方法：将股票代码（Symbol)以Symbol数据类型，建立字段后，可以用HASH方式建立分区，也可以手工根据股票代码数字特征，根据前1~3位的字符建立分区，用自定义函数建立分区（[TSDB存储引擎](https://docs.dolphindb.cn/zh/db_distr_comp/db/tsdb.html)中介绍自定义函数分区）。我们采用第二种方法。  
#### 2.3.1 代码分类器
以下是根据证券代码分区规则的分类器。分类器可以选择字段名，将所选列中的值按照最高n个进行分类，自动合并为按照上限数量的较优分类。该分类器也可以用于类似数字字符序列的值的分类，大到小排列。

In [25]:
import pandas as pd

#generate the range of values for partitions
# df: the DataFrame that store the range of values
# col: the list/column name of values
# n: the numbers of values per partition
def categorize_symbols(df, n,col='代码'):
    # Function to recursively group symbols and count items
    def recursive_groupby(symbols, level):
        grouped = symbols.groupby(symbols.str[:level])
        result = {}
        for group, items in grouped:
            if len(items) <= n:
                result[group] = len(items)
            else:
                result.update(recursive_groupby(items, level + 1))
        return result

    # Extract the 'category' column as a Series
    symbols = df[col]
    # Perform the recursive grouping and counting
    categorized_symbols = recursive_groupby(symbols, 1)

    # Convert the dictionary to a DataFrame
    ranges_df = pd.DataFrame({
        'category': list(categorized_symbols.keys()),
        'amount': list(categorized_symbols.values())
    })
    
    
    ranges_df.sort_values(by='category', inplace=True)
    ranges_df['start'] = ranges_df['category']
    ranges_df['end'] = ranges_df['category']

    # Merge categories
    merged = []
    i = 0
    while i < len(ranges_df):
        start = ranges_df.iloc[i]['start']
        total_count = ranges_df.iloc[i]['amount']
        end = ranges_df.iloc[i]['end']
        i += 1
        
        while i < len(ranges_df) and total_count + ranges_df.iloc[i]['amount'] <= n:
            total_count += ranges_df.iloc[i]['amount']
            end = ranges_df.iloc[i]['end']
            i += 1
            
        merged.append({'start': start, 'end': end, 'amount': total_count})

    merged_df = pd.DataFrame(merged)
    
    # Fill 'start' with '0' and 'end' with '9' to 6 digits
    merged_df['start'] = merged_df['start'].str.ljust(6, '0')
    merged_df['end'] = merged_df['end'].str.ljust(6, '9')
    
    # save to csv
    merged_df.index.name='partition'
    merged_df[['start','end']].to_csv('Data/partition.csv')
    return merged_df

categorized_symbols = categorize_symbols(result['df'],986,'代码')
categorized_symbols

,start,end,amount
partition,,,
0,000000,001999,516
1,002000,003999,967
2,300000,300999,947
3,301000,499999,422
4,600000,600999,766
5,601000,605999,926
6,680000,999999,813


##### 2.3.2 A股清单分区归类
按照以上分类器分类，为A股清单（含IPO日期）数据添加category分区字段，验证分区归类。

In [26]:
# add category
def addCategory(df1,df2,col='代码'):
    for index, row in df2.iterrows():
        mask = (df1[col] >= row['start']) & (df1[col] <= row['end'])
        df1.loc[mask, 'category'] = f"{index}"
    return df1
allocatedDf=addCategory(result['df'],categorized_symbols)
allocatedDf

,代码,名称,IPO,category
0,920016,N中草,2024-09-13,6
1,430198,微创光电,2020-07-27,3
2,300635,中达安,2017-03-31,2
3,300746,汉嘉设计,2018-05-25,2
4,300067,安诺其,2010-04-21,2
...,...,...,...,...
5352,300134,大富科技,2010-10-26,2
5353,688615,合合信息,2024-09-26,6
5354,603361,浙江国祥,NaT,5
5355,603091,众鑫股份,2024-09-20,5


分类器完成所有分类，没有遗漏

In [27]:
allocatedDf[allocatedDf['category'].isnull()]

,代码,名称,IPO,category


#### 2.3.3 分类函数
分类函数读取分类器生成的partition.csv文件，完成数据分区。

In [28]:
SQL1='''
def myPartitionFunc(sym) {
    pt=loadText('Data/partition.csv')
    x=select partition from pt where sym>=start and sym<=end
    return x
}
'''

#### 2.3.4 AkShare数据源获取数据和格式
我们采用东财数据源（后复权），我们将列替换为英文，并采用OHLCV字段。

In [29]:
import akshare as ak

# download akshare stock data(hfq) in the certain period, and format the data
def getFormattedAkshare(symbol,start_date,end_date=None,fq=""):
    if end_date:
        stock = ak.stock_zh_a_hist(symbol=symbol, period="daily", start_date=start_date, end_date=end_date, adjust=fq)
    else:
        stock = ak.stock_zh_a_hist(symbol=symbol, period="daily", start_date=start_date, adjust=fq)
    stock_res=stock[['日期','股票代码','开盘','收盘','最高','最低','成交量','换手率']]
    stock_res=stock_res.rename(columns={
        '日期': 'DateTime',
        '股票代码': 'Symbol',
        '开盘': 'Open',
        '收盘': 'Close',
        '最高': 'High',
        '最低': 'Low',
        '成交量': 'Volume',
        '换手率': 'TurnOver'
    })
    stock_res['DateTime'] = pd.to_datetime(stock_res['DateTime'])
    stock_res['Market'] = 'cn_stock'
    stock_res.set_index('DateTime',inplace=True)
    return stock_res
getFormattedAkshare('600019','20200101')

,Symbol,Open,Close,High,Low,Volume,TurnOver,Market
DateTime,,,,,,,,
2020-01-02,600019,5.77,5.80,5.88,5.77,1608746,0.73,cn_stock
2020-01-03,600019,5.82,5.80,5.84,5.77,810379,0.37,cn_stock
2020-01-06,600019,5.77,5.75,5.80,5.72,1051389,0.48,cn_stock
2020-01-07,600019,5.75,5.83,5.83,5.74,978161,0.44,cn_stock
2020-01-08,600019,5.80,5.74,5.80,5.73,884021,0.40,cn_stock
...,...,...,...,...,...,...,...,...
2024-09-30,600019,6.66,6.94,6.96,6.65,2510962,1.15,cn_stock
2024-10-08,600019,7.63,7.03,7.63,6.83,3003696,1.37,cn_stock
2024-10-09,600019,6.97,6.39,6.97,6.38,2368119,1.08,cn_stock


#### 2.3.5 建立分区数据库
根据symbol分类分区和日期分区建立名为stockDaily的DolphinDB。
其中，股票代码根据[数据类型](https://docs.dolphindb.cn/zh/progr/data_types.html#chap4_sect_data_type_description__section_m55_ndy_jxb) 转变为Symbol类型数据。参考[TSDB存储引擎设定自定义函数作为分区规则](https://docs.dolphindb.cn/zh/db_distr_comp/db/tsdb.html),注意，原文中在create table分区表时“partitioned by”应替换为“partitionColumns=”的形式，同时，数据库变量应该加上“`”符号。

In [30]:
import dolphindb as ddb
import dolphindb.settings as keys

s = ddb.Session()
s.connect("localhost", 8848, "admin", "123456")

SQL2='''
create database "dfs://stock"
partitioned by VALUE(1992.01.01..2030.12.31), engine='TSDB'
create table "dfs://stock"."daily"(
    dateTime DATE,
    Symbol SYMBOL,
    Open DOUBLE,
    Close DOUBLE,
    High DOUBLE,
    Low DOUBLE,
    Volume INT,
    TurnOver DOUBLE,
    Market SYMBOL
)
sortColumns=`Symbol
partitionColumns=`Symbol
'''

Failed to connect to host = localhost port = 8848 with error code 61


In [31]:
s.run(SQL1+SQL2)
s.close()

RuntimeError: Couldn't send script/function to the remote host because the connection has been closed

### 2.4 从AKShare下载数据并用国泰君安富易数据作校验
#### 2.4.1 校验函数

In [ ]:
# compare the difference of two DataFrame of the same indexes
# colList：columns list to compare
# thresh: difference threshold
def compareDiff2(tDf,vDf,thresh,colList):
    compareind=vDf.index
    targetdfcomp = tDf[tDf.index.isin(compareind)]
    
    comparedf=(targetdfcomp[colList]-vDf[colList])/(targetdfcomp[colList]+vDf[colList])*2
    
    non_null_count = comparedf.notna().sum().sum()
    diff_col_count=diff_count = (comparedf.abs() > thresh).sum()
    diff_count = diff_col_count.sum()

    null_count = comparedf.isna().sum().sum()
    diff_df=comparedf[(comparedf.abs()>thresh).any(axis=1)]
    
    if non_null_count > 0:
        commonRate = diff_count / non_null_count
        
    total_count = non_null_count + null_count
    if total_count > 0:
        totalRate = (diff_count + null_count) / total_count
    
    return {'non_null':non_null_count,'diff_count':diff_count,'null_count':null_count,'common':commonRate,'total':totalRate,'diff_col_count':diff_col_count,'df':comparedf,'diffdf':diff_df}

国泰君安富易数据作校验(取值差异0.001作为阈值，即不超过0.1%)

In [ ]:
sear='920016'
validateDf=validateDfs[sear].copy()
validateDf['Volume']=(validateDf['Volume']/100).round()
validateDf

In [ ]:
daystr=validateDf.index[0].strftime('%Y%m%d')
targetDf=getFormattedAkshare(sear,daystr)
targetDf

In [ ]:
comparelist=['Open','High','Low','Close','Volume']
compareDiff2(validateDf,targetDf,0.001,comparelist)

#### 2.4.2 下载AKShare并校验
下载和验证函数

In [ ]:
# get a single stock data from AKshare from foundamental DataFrame stocklist and verified with validateDfs Dict
#symbol:stock symbol
#validateDfs: gtja history data in the form of Dict {code of stock: DataFrame of daily history}
#foundDf: foundamental information DataFrame
# dfThresh: the threshhold of common difference between two DFs
# fq: 'qfq','hfq' and '' for forward,backward and none rights and devidend adjustments
# ATTENTION: For the variety of methods unpublished, the verification DOES NOT verify adjust data,
#   if the none adjust data pass the verification, then pass the adjust data 
def getVerifiedAKShareSingle(symbol,validateDfs,foundDf,dfThresh,fq):
    res={}
    ipoday=foundDf.loc[foundDf['代码']==symbol,'IPO']
    ipodayvalue=foundDf.loc[foundDf['代码']==symbol,'IPO'].iloc[0]
    ipodayStr=ipodayvalue.strftime('%Y%m%d')
    
    akshareDaily=getFormattedAkshare(symbol,ipodayStr)
    
    validateDf=validateDfs[symbol].copy()
    validateDf['Volume']=(validateDf['Volume']/100).round()
    
    comparelist=['Open','High','Low','Close','Volume']
    valueThresh=0.001  # threshhold of value difference from the validation benchmark. 
    comres=compareDiff2(validateDf,akshareDaily,valueThresh,comparelist)

    if comres['common']>=dfThresh:
        res['pass']=False
        res['diffdf']=comres['diffdf']
    else:
        res['pass']=True
        res['df']=akshareDaily
        res['IPOstr']=ipodayStr
    return res

In [ ]:
getVerifiedAKShareSingle('920016',validateDfs,result['df'],0.001,"")

根据A股清单遍历下载、搜集并验证数据(后复权)

In [ ]:
from datetime import datetime

# get all verified stock datas
def getVerifiedAShareAll(validateDfs,foundDf,dfThresh,fq):
    i=0
    res={}
    res['fail']=[]
    res['failDict']={}
    for index, row in foundDf.iterrows():
        if i<3:
            symbol=row['代码']
            print(symbol)
            ipo_datestr = row['IPO'].strftime('%Y%m%d')
            singleres=getVerifiedAKShareSingle(symbol,validateDfs,foundDf,dfThresh,fq)
            print('singleres',singleres)
            if singleres['pass']:
                if fq=='':
                    res[symbol]=singleres['df']
                else:
                    res[symbol]=getFormattedAkshare(symbol,res['ipostr'],fq=fq)
            else:
                res['fail'].append(symbol)
                res['failDict'][symbol]=singleres['diffdf']
        else:
            break
        i+=1
    return res

In [ ]:
getVerifiedAShareAll(validateDfs,result['df'],0.001,'hfq')

### 2.5 导入DolphinDB

In [2]:
import dolphindb as ddb
import dolphindb.settings as keys
import numpy as np
import pandas as pd

s = ddb.session(protocol=keys.PROTOCOL_DDB)
s.connect("192.168.1.113", 8848, "admin", "123456")

s.run("share table(1000:0, `sym`uuid`int128`ipaddr`blob, [SYMBOL, UUID, INT128, IPADDR, BLOB]) as t")
appender = ddb.TableAppender(tableName="t", ddbSession=s)
data = pd.DataFrame({
    'sym': ["A1", "A2", "A3"],
    'uuid': ["5d212a78-cc48-e3b1-4235-b4d91473ee87", "b93b8253-8d5e-c609-260a-86522b99864e", ""],
    'int128': [None, "073dc3bc505dd1643d11a4ac4271d2f2", "e60c84f21b6149959bcf0bd6b509ff6a"],
    'ipaddr': ["2c24:d056:2f77:62c0:c48d:6782:e50:6ad2", "", "192.168.1.0"],
    'blob': ["testBLOB1", "testBLOB2", "testBLOB3"],
})

appender.append(data)

t = s.run("t")
print(t)

Failed to connect to host = 192.168.1.113 port = 8848 with error code 51


RuntimeError: Couldn't send script/function to the remote host because the connection has been closed

In [ ]:
# 
def checkValidateDfs(validateDfs, checkdf, lastday):
    records = []
    
    for k in validateDfs.keys():
        vdf = validateDfs[k]
        msg = ''
        
        if vdf.empty:
            msg = 'no content'
        elif vdf.index.min() > checkdf.loc[checkdf['代码'] == k, 'IPO'].values[0]:
            msg = '数据较IPO日有缺口'
        elif vdf.index.min() < checkdf.loc[checkdf['代码'] == k, 'IPO'].values[0]:
            msg = '数据早于IPOday'
        elif vdf.index.max() < lastday:
            msg = '数据离终止日有缺口'
        
        records.append({'key': k, 'msg': msg})
        print(f'key: {k}, msg: {msg}')
    
    return pd.DataFrame(records)

# 示例
validateDfs = {
    'code1': vdf1,
    'code2': vdf2,
    'code3': vdf3
}
checkdf = ...
today = datetime.now().date()

df = checkValidateDfs(validateDfs, checkdf, today)
print(df)


## 附件：工具集和参考

#### 数据处理过程参考：
涨停板敢死队.ipynb

#### 单个表结构查询

In [ ]:
import sqlite3

#get the field list of a table
def getStruct(filelink,table_name):
    conn = sqlite3.connect(filelink)
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name});")
    fields = cursor.fetchall()
    conn.close()
    return [field[1] for field in fields]

getStruct(filelink,f'gtja_{symbol}')

#### AKShare查询个股信息和IPO等

In [ ]:
b_df=ak.stock_individual_info_em('301551')
b_df

#### Dolphin DB 数据维护工具
删除指定表

In [82]:
s = ddb.Session()
s.connect("localhost", 8848, "admin", "123456")
SQL='''
dbName="dfs://stock"
dropDatabase(dbName)
'''
s.run(SQL)
s.close()

In [ ]:
查询所有表
